# Zpracování MARC souborů pomocí knihovny Pymarc

Tento notebook vás provede procesem zpracování MARC souborů za použití knihovny Pymarc. MARC (Machine-Readable Cataloging) je standardní formát pro záznamy knihovních katalogů a metadata. Tento notebook je určený jak pro začátečníky, tak pro ty, kteří se chtějí seznámit s knihovnou Pymarc a zpracováním MARC dat v Pythonu.

## Předpoklady

Tento notebook nepředpokládá hluboké znalosti Pythonu, ale základní znalost programování bude užitečná.<br> 
Ukážeme si, jak načíst marcové soubory, jak procházet jejich záznamy, jak extrahovat potřebná data a jak je uložit do CSV a Excel souborů.

## Struktura notebooku

Tento notebook je rozdělen do několika částí:

0. **Příprava**: Přidáme potřebné knihovny, které budeme používat ke zpracování marcového souboru. 

1. **Načtení MARC souboru**: Ukážeme si, jak načíst marcový soubor a vypsat jednotlivé záznamy.

2. **Získání dat**: Naučíme se, jak získat specifická data z marcového záznamu, jako jsou názvy, autoři a žánry.

4. **Export do CSV a Excelu**: V závěrečné části si ukážeme, jak uložit získaná data do CSV a Excel souborů pro další analýzu.


## Další zdroje

- [LearnPython.org](https://www.learnpython.org/): Tento online kurz nabízí výuku jazyka Python pro začátečníky i pokročilé. Může být užitečným zdrojem pro ty, kteří chtějí rozšířit své znalosti Pythonu.



### 0. Příprava 
Jako první si musíme nainstalovat knihovny, se kterými budeme pracovat. Knihovny jsou balíčky funkcí, které nejsou součástí základu jazyka Python. <br>
Knihovny nainstalujeme pomocí příkazu `%pip install <jmeno_knihovny>` . Pak je do našeho notebooku přidáme pomocí příkazu `import <jmeno_knihovny> (as alias)`. K funkcím knihovny se pak přistupuje `jmeno_knihovny.jmeno_funkce` <br> 
Pokud z knihovny chceme využít pouze jednu funkci, přidáme ji pomocí `from <jmeno_knihovny> import <jmeno_funkce>`

In [1]:
# Prikaz ktery naistaluje knihovny
%pip install pandas 
%pip install pymarc 
%pip install openpyxl

# Prikaz ktery knihovny prida 
import pandas as pd
from pymarc import MARCReader



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 1. Načtení MARC souboru

Naše data jsou uložena v knihovním formátu MARC s koncovnkou .mrc. Pro práci budeme používat funkci `MARCReader` z knihovnu `pymarc`, která data ze souboru načte a záznamy rozdělí, takže k nim pak můžeme přistoupit jednotlivě. <br>  

#### 1.1 Výpis záznamu

Vypíšeme si, jak takový záznam vypadá. <br>
Nejprve si určíme, kolikátý záznam chceme vypsat. To definujeme pomocí proměnné `ith`.<br> 
Pak otevřeme marcový soubor a postupně ho projdeme. Funkce `enumerate()` nám záznamy navíc očísluje od 0. Díky tomu budeme moct vypsat i-tý záznam.<br>
Abychom nemuseli procházet celý soubor, napíšeme slovo `break`, které ukončí procházení souboru.


In [2]:
# Cesta k marcovemu dokumentu
database = 'data/ucla/ucla_smz.mrc'

# i-ty zaznam, ktery chceme vypsat
ith = 5

# Otevreni souboru
with open(database, 'rb') as data:
    # Nacteni marcu
    reader = MARCReader(data)

    # Postupne prochazime a pocitame zaznamy v nactenem souboru 
    for i,record in enumerate(reader):        
        
        # Pokud je i nas zaznam, vypiseme ho 
        if i == ith:
            # Vypsani marcovych zaznamu
            print(record)

            # Chceme vypsat pouze jeden zaznam, tak po prvnim vypsani zavolame break
            break


=LDR  00878nab a22003134a 4500
=001  001457857
=003  CZ\PrUCL
=005  20220321104842.0
=008  090923e197907\\xr\\\\\\\\\\\\|||\||cze\d
=035  \\$a(ISIS-B70-MFN)78976
=035  \\$a(ISIS-B70-ID)795002
=040  \\$aABB060$bcze
=080  \\$a7.01$2MRF
=080  \\$a(0:82-4)$2MRF
=100  1\$aVašíček, Zdeněk,$d1933-2011$7jk01141688$4aut
=245  10$aArcheologie času /$cZdeněk Vašíček.
=500  \\$aPodnázev zdrojového dokumentu: (London, Index on Censorship).
=520  2\$aEsej.
=599  \\$aCLB-CPK
=650  07$aestetika$7ph117189$2czenas
=655  \7$aeseje$7fd132213$2czenas
=773  0\$tSpektrum$gR. 1979, č. 2, červenec, s. 14-31$q1979:2<14$9197907
=964  \\$aB70
=964  \\$aSMZ
=964  \\$aCLE
=OWN  \\$aUCLA
=SIF  \\$afk
=910  \\$aABB060
=ZAZ  \\$d197907$s1979$z2$l14



Vidíme, že marcový soubor má jasnou strukuru. Má několik polí, která jsou označena zpravidla třemi číslicem, případně třemi písmeny. Každý kód má svou vnitřní logiku, např. pole pro věcné popisy vždy začínají číslicí ``6XX``. <br>
Za číslem - tagem pole se obvyke nachází dva indikátory. Pokud indikátor není definovaný, píše se místo něj zpětné lomítko (\\). <br>
Většina polí se dělí na podpole. Nachází se za dolarem (``$``) a označují se buď jedním písmenem, nebo číslicí.  



<div class='alert alert-block alert-info'>
    <b>Try It!</b>  Pomocí parametru ith nastavíme kolikátý záznam chceme vypsat (indexování začíná od 0). Pokud bychom chtěli vypsat všechny záznamy do i-tého záznamu, změníme `if i == ith:` na `if i <= ith:` a smažeme příkaz `break`.
</div>



#### 1.2 Výpis jednotlivých polí

Pro práci s databází pravděpodobně nebudeme potřebovat všechna pole, proto si teď ukážeme, přistupovat k jednotlivým polím v záznamu. <br>
Vypíšeme si jen číslo záznamu, název, autora a žánr. K některým polím se můžeme dostat přes tečkovou notaci (`record._`), k ostatním musíme přistupovat přes funkci `get_fields(<cislo pole>)`, která vrátí všechna pole s číslem v závorce.  

In [25]:
ith = 5

# Otevreni souboru
with open(database, 'rb') as data:
    # Nacteni marcu
    reader = MARCReader(data)

    # Postupne prochazime a pocitame zaznamy v nactenem souboru 
    for i, record in enumerate(reader):

        if i == ith:
            # Vypsani marcoveho zaznamu
            # K nekterym polim muzeme pristupovat pres teckovou notaci, tedy record.leader nebo record.title 
            print("Záznam: " + record.leader)
        
            # Pred vypsanim je standardem se nejprve podivat, zda zaznam existuje (tedy ze neni None). 
            # Pokud bychom se totiz snazili vypsat None hodnotu, kod by vyhodil error. 
            if record.title is not None:
                print("Název: " + record.title)
            if record.author is not None:
                print("Autor: " + record.author)
            
            # Pokud chceme zjistit, zda pole ktere nema teckovou notaci neni None, je potreba zavolat funci .get_fields()     
            if record.get_fields('655') is not None:     
                # K polim, ktere nemaji teckovou notaci, se pristupuje pres zavorky 
                print("Žánr: " + record['655']['a'])
            break        

Záznam: 00878nab a22003134a 4500
Název: Archeologie času /
Autor: Vašíček, Zdeněk, 1933-2011 jk01141688 aut
Žánr: eseje


#### 1.3 Počet záznamů

Také by nás mohlo zajímat, kolik záznamů je v dané databázi. K tomu si vytvoříme samostatnou funkci `number_of_records(database)`, kterou pak jen zavoláme. Vlastní funkce definujeme pomocí slova `def`<br> 
Funkce jako vstup bere cestu k marcové databázi. Uvnitř funkce si otevře, projde a při každém záznamu si připočítá jedničku k počítadlu `counter`.  

In [26]:
# Vlastni funkce definujeme pomoci slova def
def number_of_records(database):
    
    with open(database, 'rb') as data:
        # Nacteni marcu
        reader = MARCReader(data)
        # Vytvorime pocitadlo, ke kteremu pri kazdem zaznamu pricteme jednicku
        counter = 0
        # Vzhledem k tomu, ze record nepotrebujeme, tak muzeme pouzit podtrzitko (_), ktere hodnotu bude ignorovat 
        for _ in reader:
            counter += 1
    
    # Pokud chceme nejakou hodnotu vratit, pouzijeme slovo return        
    return counter 

print("V databázi je " + str(number_of_records(database)) + " záznamů.")        

V databázi je 21561 záznamů.


### 2. Získání dat 

S marcovým dokumentem se nepracuje příliš dobře, proto je lepší si data uložit do jednodušší tabulky. V této fázi si musíme ujasnit, jaká data budeme chtít. V našem příkladu budeme chtít uložit název, autora, autorův kód, rok vydání a pak pole '``600 $a``','``650 $a``', '``655 $a``' a '``773 $t``'. <br>

#### 2.1 Marcová pole

Veškeré záznamy začínající číslem 6XX jsou věcné údaje o záznamu. Tato pole se mohou opakovat. <br>     
Pod polem '600' se skrývají osoby, o kterých záznam je nebo případně osoby, kterým je záznam dedikován. <br> 
Pod polem '650' se nacházejí věcné termíny/téma, tzn. o čem záznam je. <br>
Pod polem '655' pak najdeme žánr daného záznamu. Na rozdíl od polí '600' a '650' by pole '655' mělo být přítomné u každého záznamu. <br>
Záznamy začínající čísly 76X - 78X se nazývají propojovací pole a slouží pro zápis zdrojového (773) nebo recenzovaného (787) dokumentu. <br>



In [3]:
# i-ty zaznam, ktery chceme vypsat
ith = 10

# Otevreni souboru
with open(database, 'rb') as data:
    # Nacteni marcu
    reader = MARCReader(data)

    # Postupne prochazime a pocitame zaznamy v nactenem souboru 
    for i,record in enumerate(reader):        
        
        # Pokud je i nas zaznam, vypiseme ho 
        if i == ith:
            
            print("Záznam: " + record.leader)
            
            # Pokud pole existuje, vypiseme jeho obsah
            if record.get_fields('600') is not None:   
                # Protoze se pole muze opakovat, projdeme a vypiseme vsechna pole  
                for field in record.get_fields('600'): 
                    print("Věcné jméno: " + field['a'])
                
                
            # Pokud pole existuje, vypiseme jeho obsah    
            if record.get_fields('650') is not None:    
                # Protoze se pole muze opakovat, projdeme a vypiseme vsechna pole
                for field in record.get_fields('650'): 
                    print("Téma: " + field['a'])
            
            # Pokud pole existuje, vypiseme jeho obsah
            if record.get_fields('655') is not None: 
                # Protoze se pole muze opakovat, projdeme a vypiseme vsechna pole   
                for field in record.get_fields('655'): 
                    print("Žánr: " + field['a'])

            # Chceme vypsat pouze jeden zaznam, tak po prvnim vypsani zavolame break
            break


Záznam: 00940nab a22003014a 4500
Věcné jméno: Stankovič, Andrej,
Téma: čeští spisovatelé
Žánr: bio-bibliografické poznámky


#### 2.2 Selekce polí

Pro ukládání máme připravenou funkci `save_to_dict(record, dictionary, field_list)`, která nám jeden záznam (`record`) uloží do dictionary (struktury v Pythonu) `marc_dictionary`. <br>
Struktura dictionary je tvořena dvojicí klíč - hodnota. K hodnotě se dostaneme skrz klíč v hranatých závorkách. `dict[<klic>] = <hodnota>`<br> 
Protože není strategické si do tabulky ukládat všechna pole, podpole a indikátory, funkci `save_to_dict` kromě záznamu předáme také list polí `field_list`, která chceme uložit. <br>

Jelikož se některá pole (např. 700) mohou opakovat, je dobré si hodnoty za každé pole uložit do listu (kolekce v jazyce Python). List je seznam hodnot, např. textu - stringů, čísel - integerů nebo float atd. <br>
Kvůli obecnosti si do listu uložíme veškeré hodnoty. Je jednodušší, když pracujeme pouze s jedním typem (např. list nebo string), než když jsou některé hodnoty v listu a některé jsou pouze string. Pokud jsme si jisti, že v původních záznamech se pole nemůže opakovat, můžeme hodnoty uložit samostatně mimo list. <br>    

In [4]:
def save_to_dict(record, marc_dictionary, field_list):
    if not record is None:
        try:
            # Prochazeni skrz tuples v listu field_list
            for field_tags in field_list:
                # Nazev klice ve slovniku
                dict_key_name =  field_tags[0]

                # Tag pole
                tag =  field_tags[1]

                # Tag podpole
                subfield_tag =  field_tags[2]
                
                # List do ktereho pridame hodnoty a nasledne pridame do slovniku
                dict_add_list = []
                
                # Prochazeni pres vsechna pole s tagem 'tag'
                for field in record.get_fields(tag):
                    
                    # Pokud pole nema zadna podpole, pridame cele pole do listu dict_add_list
                    if subfield_tag is None:
                        dict_add_list.append(field.data) 
                    
                    # Pokud subtag je instance slice, tedy to znamena, ze chceme jen nejakou cast pole, ktera neni definovana subpolem,
                    # pridame cast pole do slovniku dict_add_list    
                    elif isinstance(subfield_tag, slice):
                        dict_add_list.append(field.data[subfield_tag])     
                    
                    # Pokud pole obsahuje podpole, pridame do slovniku dict_add_list jen podpole
                    elif '$'+subfield_tag in str(field):  
                        dict_add_list.append(str(field[subfield_tag]))

                # Do klice z tuplu pridame cely list dict_add_list         
                marc_dictionary[dict_key_name].append(dict_add_list)
        except Exception as error:
            print("Exception: " + type(error).__name__)  
            print("LDR: " + str(record.leader))   
    return marc_dictionary 

print("Funkce uložena.")

Funkce uložena.


Naši funkci teď využijeme k tomu, abychom si data z marcového souboru vytáhli. Nejprve si do listu `field_list` napíšeme, jaké hodnoty chceme. <br>
`field_list` sestává z tuplů (kolekce v jazyce Python), kde každý tuple vypadá následovně. Na první pozici je název klíče, pod kterým se rozhodneme pole uložit, na druhé pozici tag pole a na třetí tag podpole, např. ('author', '100', 'a'). <br>
Pak si vytvoříme proměnnou  `marc_dictionary`, do kterého budeme postupně hodnoty přidávat pomocí naší funkce `save_to_dict`. Klíče `marc_dictionary` jsou první hodnoty z tuplů ve `field_list`, hodnoty pak data z marcového záznamu v jednom listu.<br>
Nakonec data převedeme do datové struktury DataFrame (která je podobná např. excelovské tabulce), se kterou je mnohem jednodušší pracovat. <br>
Řádky v DataFramu reprezentují jednotlivé záznamy, sloupce pak jeden typ (např. jméno autora).

In [10]:
with open(database, 'rb') as data:
    reader = MARCReader(data)
    # List poli, ktere si chceme ulozit
    field_list = [('title', '245', 'a'),
                ('author', '100', 'a'),
                ('author code', '100', '7'),
                # Rok je schovany v poli 008 na 8. az 11. miste, proto vyuzijeme funkci slice
                ('year', '008', slice(7,11, None)), #7-11
                ('figures', '600', 'a'),
                ('description', '650', 'a'),
                ('genre', '655', 'a'),
                ('magazine', '773', 't')]
    
    # Struktura dictionary, do ktere data budeme ukladat
    marc_dictionary = {}
    
    # Postupne projdeme vsechny tuples ve field_list
    for t in field_list:
        
        # Nazev klice je v listu poli na prvnim miste    
        dict_key_name = t[0]
        
        # Do dictionary pridame nazev klice a prozatim prazdny seznam
        marc_dictionary[dict_key_name] = []
    
    # Projdeme vsechny zaznamy v databazi  
    for record in reader:
        
        marc_dictionary = save_to_dict(record, marc_dictionary, field_list)

# Marc_dictionary prevedeme do DataFramu        
df = pd.DataFrame.from_dict(marc_dictionary)

print("Marcový soubor uložen do DataFramu df.")

Marcový soubor uložen do DataFramu df.


In [11]:
# Vypise poslednich 5 zaznamu v DataFramu
df.tail()

,title,author,author code,year,figures,description,genre,magazine
21556,[Není špatného tabáku ani ošklivých žen /],"[Infeld, Leopold,]",[jn20001227801],[1979],"[Einstein, Albert,]","[fyzici, Nobelova cena (ocenění), osamělost, i...",[vzpomínky],[Acta incognitorum [samizdat]]
21557,[Bude religiózní nebo /],"[Jirous, Ivan Martin,]",[jn19981001568],[1980],"[Němec, David,, Zajíček, Pavel,]","[výtvarní umělci, výtvarné umění, pop art, kon...",[články],[Magorův zápisník I : duben 1979 - duben 1980 ...
21558,"[Zasadil jsem vám osiku, pane doktore! /]","[Jirous, Ivan Martin,]",[jn19981001568],[1980],"[Bondy, Egon,, Batrouvid, Orwell, George,]","[undergroundová kultura, čeští básníci, čeští ...",[polemiky],[Magorův zápisník I : duben 1979 - duben 1980 ...
21559,[Krása bude křečovitá nebo nebude vůbec /],"[Jirous, Ivan Martin,]",[jn19981001568],[1980],"[Zajíček, Pavel,, Hlavsa, Milan,, Kabeš, Jiří,]","[undergroundová kultura, alternativní hudba, a...",[reportáže],[Magorův zápisník I : duben 1979 - duben 1980 ...
21560,[Stalo se něco hrozného /],"[Jirous, Ivan Martin,]",[jn19981001568],[1980],"[Wilde, Oscar,, Kordík, Josef,, Soukup, Karel,]","[divadelní představení, anglické drama, underg...",[recenze],[Magorův zápisník I : duben 1979 - duben 1980 ...


#### 2.3 Úprava dat 

Vidíme, že jména za sebou mají přebytečnou čárku, kterou jednoduše odstraníme. Stejně tak názvy přebytečné lomítko. Díky funkcím `apply()` a `lambda` můžeme změnit všechny hodnoty ve sloupci pomocí jednoho řádku kódu. Ve funkci `lambda` si definujeme, jak data chceme upravit. Funkcí `apply()` použijeme funkci `lambda` na všechny hodnoty. <br> 
Všechna data jsou uložena v listu, proto musíme projít všechny hodnoty v list zvlášť. <br>  
Funkce vezme jednotlivé listy ve sloupci a zjistí, zda list není prázdný ( tedy se zeptá, zda je velikost listu větší než 0 `len(y) > 0`). Pokud není, funkce projde hodnoty v listu a uloží si je bez koncové čárky. To udělá pomocí funkce `strip(' ,')`, která případně odstraní koncové čárky nebo mezery u jmen. U názvů použijeme funkci `strip(' /')`, která nás zbaví přebytečného lomítka. Pokud je prázdný, funkce list nijak nezmění.   

In [12]:
# U jmen si chceme ulozit jmeno a prijmeni bez koncove carky ',', ktera je na konci stringu
df['figures'] = df['figures'].apply(lambda x: [y.strip(' ,') if len(y) > 0  else y for y in x])  
df['author'] = df['author'].apply(lambda x: [y.strip(' ,') if len(y) > 0 else y for y in x]) 

# Nazev si chceme ulozit bez lomitka '/', ktere je na konci stringu
df['title'] = df['title'].apply(lambda x: [y.strip(' /') if len(y) > 0 else y for y in x])  

# Vypise poslednich 5 zaznamu v DataFramu
df.tail()

,title,author,author code,year,figures,description,genre,magazine
21556,[Není špatného tabáku ani ošklivých žen /],"[Infeld, Leopold]",[jn20001227801],[1979],"[Einstein, Albert]","[fyzici, Nobelova cena (ocenění), osamělost, i...",[vzpomínky],[Acta incognitorum [samizdat]]
21557,[Bude religiózní nebo /],"[Jirous, Ivan Martin]",[jn19981001568],[1980],"[Němec, David, Zajíček, Pavel]","[výtvarní umělci, výtvarné umění, pop art, kon...",[články],[Magorův zápisník I : duben 1979 - duben 1980 ...
21558,"[Zasadil jsem vám osiku, pane doktore! /]","[Jirous, Ivan Martin]",[jn19981001568],[1980],"[Bondy, Egon, Batrouvid, Orwell, George]","[undergroundová kultura, čeští básníci, čeští ...",[polemiky],[Magorův zápisník I : duben 1979 - duben 1980 ...
21559,[Krása bude křečovitá nebo nebude vůbec /],"[Jirous, Ivan Martin]",[jn19981001568],[1980],"[Zajíček, Pavel, Hlavsa, Milan, Kabeš, Jiří]","[undergroundová kultura, alternativní hudba, a...",[reportáže],[Magorův zápisník I : duben 1979 - duben 1980 ...
21560,[Stalo se něco hrozného /],"[Jirous, Ivan Martin]",[jn19981001568],[1980],"[Wilde, Oscar, Kordík, Josef, Soukup, Karel]","[divadelní představení, anglické drama, underg...",[recenze],[Magorův zápisník I : duben 1979 - duben 1980 ...


### 3. Export do CSV a excelu

V následujícím kroku si budeme chtít data exportovat do formátu CSV. Jelikož CSV tabulka nepracuje dobře s listy, hodnoty v listu spojíme středníkem do jednoho stringu pomocí funkce `join()`. K tomu opět využijeme lambda funkci. <br>

###### Funkce `join()` může spojit i jednotlivá písmena ve stringu. Abychom předešli tomu, že mezi každým písmenkem budeme mít středník, v lambda funkci pomocí `isinstance()` otestujeme, zda jsou data opravdu v listu. 


In [9]:
# Aby se nam list hodnot lepe ukladal, vytvorime z listu jeden string a jednotlive elementy spojime strednikem ';' 
for column in df.columns:
    df[column] = df[column].apply(lambda x: ';'.join(x) if isinstance(x, list) else x )

df.tail()

,title,author,author code,year,figures,description,genre,magazine
21556,Není špatného tabáku ani ošklivých žen /,"Infeld, Leopold",jn20001227801,1979,"Einstein, Albert",fyzici;Nobelova cena (ocenění);osamělost;inter...,vzpomínky,Acta incognitorum [samizdat]
21557,Bude religiózní nebo /,"Jirous, Ivan Martin",jn19981001568,1980,"Němec, David;Zajíček, Pavel",výtvarní umělci;výtvarné umění;pop art;koncert...,články,Magorův zápisník I : duben 1979 - duben 1980 [...
21558,"Zasadil jsem vám osiku, pane doktore! /","Jirous, Ivan Martin",jn19981001568,1980,"Bondy, Egon;Batrouvid;Orwell, George",undergroundová kultura;čeští básníci;čeští pro...,polemiky,Magorův zápisník I : duben 1979 - duben 1980 [...
21559,Krása bude křečovitá nebo nebude vůbec /,"Jirous, Ivan Martin",jn19981001568,1980,"Zajíček, Pavel;Hlavsa, Milan;Kabeš, Jiří",undergroundová kultura;alternativní hudba;alte...,reportáže,Magorův zápisník I : duben 1979 - duben 1980 [...
21560,Stalo se něco hrozného /,"Jirous, Ivan Martin",jn19981001568,1980,"Wilde, Oscar;Kordík, Josef;Soukup, Karel",divadelní představení;anglické drama;undergrou...,recenze,Magorův zápisník I : duben 1979 - duben 1980 [...


Nakonec DataFrame uložíme do CSV a Excelu. 

In [33]:
out_csv = 'data/csv/out_smz.csv'

# DataFrame ulozime do formatu CSV
df.to_csv(out_csv, encoding = 'utf8', sep = ",", index=False)   

print("Data uložena do csv.")

out_excel = 'data/out_smz.xlsx'

# Soubor muzeme ulozit i jako excelovy soubor, se kterym dal muzeme pracovat v Excelu
df.to_excel(out_excel,  index=False) 

print("Data uložena do xlsx.")

Data uložena do csv.
Data uložena do xlsx.
